# Publication database
Create a database that tracks publications and their authors in order

In [ ]:
import datajoint as dj

In [ ]:
schema = dj.schema('dimitri_publications')

In [ ]:
@schema
class Publication(dj.Manual):
    definition = """
    pub_id : int 
    ---
    title : varchar(1000)
    publish_date  : date 
    author : varchar(1000)
    """

In [ ]:
import faker
faker = faker.Faker()

In [ ]:
for i in range(30):
    Publication.insert1(dict(
        pub_id=i, title=faker.sentence(), 
        publish_date=faker.date_this_decade(), author=faker.name()), skip_duplicates=True)

In [ ]:
Publication()

# New design. Many-many publication/author

In [ ]:
schema.drop()
schema = dj.schema('dimitri_publications')

In [ ]:
@schema 
class Author(dj.Manual):
    definition = """
    author_id : int 
    ---
    first_name : varchar(100)
    last_name : varchar(100)
    """

In [ ]:
for i in range(100):
    Author.insert1(dict(author_id=i, first_name=faker.first_name(), last_name=faker.last_name()))  

In [ ]:
# same thing but faster
Author.insert((
    dict(author_id=i, first_name=faker.first_name(), last_name=faker.last_name()) for i in range(100))
    , skip_duplicates=True)

In [ ]:
Author()

In [ ]:
@schema
class Publication(dj.Manual):
    definition = """
    pub_id : int 
    ---
    title : varchar(1000)
    publish_date  : date 
    """

In [ ]:
Publication.insert([dict(
    pub_id=i, title=faker.sentence(), 
    publish_date=faker.date_this_decade()) for i in range(30)], skip_duplicates=True)

In [ ]:
Publication()

In [ ]:
@schema
class PubAuthor(dj.Manual):
    definition = """
    -> Publication
    -> Author
    """

In [ ]:
authors = Author.fetch("KEY")
pubs = Publication.fetch("KEY")

In [ ]:
import random

In [ ]:
for pub in pubs:
    for auth in random.choices(authors, k=random.randint(1, 3)):
        PubAuthor.insert1({**pub, **auth}, skip_duplicates=True)

In [ ]:
PubAuthor()

In [ ]:
Publication * PubAuthor * Author & 'pub_id=6'

### Modify the design to specify the order of authors on the publication

In [ ]:
PubAuthor.drop()

In [ ]:
@schema
class PubAuthor(dj.Manual):
    definition = """
    -> Publication
    -> Author
    ---
    author_order : tinyint 
    unique index(pub_id, author_order)
    """

In [ ]:
for pub in pubs:
    for author_order, auth in enumerate(random.choices(authors, k=random.randint(1, 3))):
        PubAuthor.insert1(dict(**pub, **auth, author_order=author_order+1), skip_duplicates=True)

In [ ]:
PubAuthor()

In [ ]:
@schema
class PubAuthor(dj.Manual):
    definition = """
    -> Publication
    author_order : tinyint 
    ---
    -> Author
    unique index (pub_id, author_id)
    """

### Think next

Design a database with
- Cities
- States
- State capitals

1. Cities belong to states
2. A state capital is a city 
3. A capital must be in the same state 
4. Only one city can be capital in each state